## Face Recognition – Unlock Your Computer With Your Face!

In [ ]:
""""
Project Description:

In this Project we have followed three steps 
1.Create traning Data
2.Train model
3.Run our facial recognition

Model is trained on my face so it will detect my face and send me the message on
Whatsap,Telegram along with email and sms.

For sending the message on whatsap we have used twilio,a third party app which gives us credentials and a number.
Same for the sms and for email mailgun has been used, it also provides us the credentials and an email.
for telegram we have created a bot on telegram by taking the help of BotFather and created one channel and make 
our bot and admin of that channel.We will get the message on that channel only.

If it will detect someone else face then it will give activate the buzzer and buzz for 5 secs.It is the concept of IoT.
It will also require the api key and device id of wifi module.
Also it will launch the EC2 instance on the aws cloud and 5gb volume and attach it with the instance.
For launching an instance and volume we have used terrform. From one single code everything done in secs.

""""



### Step 1 - Create Training Data

In [1]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-629aa437f198>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


### Step 2 - Train Model

In [2]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


### Step 3 - Run Our Facial Recognition

In [4]:
import cv2
import numpy as np
import os
from boltiot import Email, Sms, Bolt
import json, time
import requests
from datetime import datetime
from twilio.rest import Client

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


NAME = "Mahek Batra"
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

#twilio is used for getting the credentials and number in a free trail account.
def sms():
    SID = 'AC1f9XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
    AUTH_TOKEN = 'ee3b4c626XXXXXXXXXXXXXXXXXXXXXX'
    FROM_NUMBER = +18XXXXXXX
    TO_NUMBER =+91937XXXXXXX
    
    sms = Sms(SID, AUTH_TOKEN, TO_NUMBER, FROM_NUMBER)
    
    print("Making request to Twilio to send a SMS")
    response = sms.send_sms("Face detected, This is face of " + NAME + " Detected at " + dt_string)
    print("Response received from Twilio is: " + str(response))
    print("Status of SMS at Twilio is :" + str(response.status))
    

#mailgun is used for gettting the credentials and sender's email 
def mail():
    MAILGUN_API_KEY = "822c6XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    SANDBOX_URL=  "sandboxXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" 
    SENDER_EMAIL = 'Face_Detector@' + SANDBOX_URL
    RECIPIENT_EMAIL = "batraXXXXXXXXXXXXXX"
    
    mailer = Email(MAILGUN_API_KEY, SANDBOX_URL, SENDER_EMAIL, RECIPIENT_EMAIL)
    
    print("Making request to Mailgun to send an email")
    response = mailer.send_email("Your Face detected", "This is face of " + NAME + " Detected at " + dt_string)
    response_text = json.loads(response.text)
    print("Response received from Mailgun is: " + str(response_text['message']))
  
    """
    A Bot is created on telegram wit the help of BotFather and a channel is created and made the bot admin of that
    channel and all the messages will get recieved on that channel only.
    
    """
    
def telegram():
    telegram_bot_id = "bot177XXX:XXXXXXXXXXXXXXXXXXXXXXXXXX"
    telegram_chat_id = "@face_detection_xxxxx"
    
    """Sends message via Telegram"""
    url = "https://api.telegram.org/" + telegram_bot_id + "/sendMessage"
    telegram_message = "Face Detected, this is face of " + NAME + " Detected at " + dt_string
    data = {
        "chat_id": telegram_chat_id,
        "text": telegram_message
    }
    response = requests.request(
        "POST",
        url,
        params=data
    )
    
#Twilio is used for sending the message  
def whatsapp():
    account_sid = 'AC1fXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' 
    auth_token = 'ee3b4XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
    client = Client(account_sid, auth_token) 
 
    message = client.messages.create( 
                              from_='whatsapp:+1415XXXXXX', 
                              body='Face Detected, this is face of '+ NAME + " Detected at " + dt_string,      
                              to='whatsapp:+91937XXXXXXX'
                          ) 
 
    print(message.sid)

#Terraform is used for creating an instance on the aws cloud and 5gb volume also attach it with the instance
def aws():
    os.system("terraform init")
    os.system("terraform apply --auto-approve")
    print("Ec2 instance created\nVolume created\n 5 GB Volume attached to instance")
 
 #Bolt:IoT 
def buzzer():
    API_KEY = 'b47XXXXXXXXXXX'
    DEVICE_ID = 'BOLTXXXXXXXX'
    
    mybolt = Bolt(API_KEY, DEVICE_ID)
    
    buzzer_response = mybolt.digitalWrite(0,'HIGH')
    time.sleep(5)
    buzzer_response = mybolt.digitalWrite(0,'LOW')
    
    
# Open Webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        
        if confidence > 90:
            cv2.putText(image, "Hey Mahek", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
           
            """
            calling of sms ,whatsapp,telegram and mail function which will send email,whatsap messaage,
            telegram message and sms with proper date and time
            
            """"          
            sms()
            mail()
            telegram()
            whatsapp()
            
            break
         
        else:
            cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
          
            #calling of aws and buzzer function
            #A buzzer sound gets produced and instance launched on aws cloud
            
            aws()
            buzzer()
            
            break
    
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
    
   
    
    if cv2.waitKey(10) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()

<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-4-037f57a8c1a6>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Making request to Twilio to send a SMS
Response received from Twilio is: <Twilio.Api.V2010.MessageInstance account_sid=AC1f9d241e0482ff96339499bd6297b9fc sid=SM1ca27d3b972f4a49a298692b70ca8874>
Status of SMS at Twilio is :queued
Making request to Mailgun to send an email
Response received from Mailgun is: Queued. Thank you.
SM6f0c69c9d8ee4fd598185d1a86c158e7
